# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
pip install azureml-sdk==1.59.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.59.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/automl-env/lib/python3.9/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install 'azureml-sdk[notebooks]'

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install azureml-sdk[automl]

Note: you may need to restart the kernel to use updated packages.


In [5]:
#from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
from azureml.core import Workspace, Experiment

Exp_name="Proj3HD"
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=Exp_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: jjprivat
Azure region: eastus2
Subscription id: 7a7e8720-181a-4ad1-ba1c-b77bc59956d1
Resource group: jjprivat


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Using existing cluster")
except ComputeTargetException:
    print("No existing cluster found, creating a cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

No existing cluster found, creating a cluster
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
dataset = Dataset.get_by_name(ws, name='proj3data')


In [12]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
src = ScriptRunConfig(source_directory='./',
                script='train.py',
                compute_target=cluster_name,
                environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=20,
                max_concurrent_runs=4)

In [13]:
#TODO: Submit your experiment
from collections.abc import Mapping
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_717a202b-ab89-47a4-b485-47e364c8cda8
Web View: https://ml.azure.com/runs/HD_717a202b-ab89-47a4-b485-47e364c8cda8?wsid=/subscriptions/7a7e8720-181a-4ad1-ba1c-b77bc59956d1/resourcegroups/jjprivat/workspaces/jjprivat&tid=4ea95936-d59c-4552-8f88-0f61cc4b5a08

Streaming azureml-logs/hyperdrive.txt

[2025-05-21T08:36:05.2231510Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 20, only 12 jobs will be generated 
[2025-05-21T08:36:06.7939271Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-21T08:36:06.9924961Z][SCHEDULER][INFO]Scheduling job, id='HD_717a202b-ab89-47a4-b485-47e364c8cda8_0' 
[2025-05-21T08:36:07.0639917Z][SCHEDULER][INFO]Scheduling job, id='HD_717a202b-ab89-47a4-b485-47e364c8cda8_1' 
[2025-05-21T08:36:07.1073672Z][SCHEDULER][INFO]Scheduling job, id='HD_717a202b-ab89-47a4-b485-47e364c8cda8_3' 
[2025-05-21T08:36:07.0654690Z][SCHEDULER][INFO]Scheduling job, id='HD_717a202b-ab89-47a4-b485-47e364c8cda8_2' 
[2025-05-21T08:36:07.3711140Z][SCHEDU

{'runId': 'HD_717a202b-ab89-47a4-b485-47e364c8cda8',
 'target': 'Cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-05-21T08:36:04.97208Z',
 'endTimeUtc': '2025-05-21T08:54:12.789161Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7ee0fef2-4880-4ec0-867f-288be79a2b9a',
  'user_agent': 'python/3.9.21 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.59.0',
  'best_child_run_id': 'HD_717a202b-ab89-47a4-b485-47e364c8cda8_1',
  'score': '0.8933333333333333',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_717a202b-ab89-47a4-b485-47e364c8cda8_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_model=best_run.register_model(model_name = 'hd_best_model.pkl', model_path='outputs/')

Run(Experiment: Proj3HD,
Id: HD_717a202b-ab89-47a4-b485-47e364c8cda8_1,
Type: azureml.scriptrun,
Status: Completed)


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [15]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '5', '--max_iter', '100']


In [16]:
print(best_run.get_metrics())

{'Max iterations:': 100, 'Regularization Strength:': 5.0, 'Accuracy': 0.8933333333333333}


## Model Deployment - NOT incl. go to AUTOML

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

AutoML was the best model, therefor the HD was not deployed to the endpoint